<a href="https://colab.research.google.com/github/jko6422/testfile/blob/main/test_06_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특수문자 손글씨 인식 프로그램

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/colab/Minst_test/

데이터 세트 제작 파트

In [ ]:
#필요 함수 import
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

"""
1. 데이터 세트 만들기 파트.
원활한 이미지 인식을 위한 준비 작업.
"""

#이미지 파일 경로 지정. 이미지는 camscanner로 보정 후 드라이브에 업로드한다.
filename = '/content/drive/MyDrive/colab/Minst_test/data/testfile1.jpg'

#그레이 스케일로 변환 
src = cv2.imread(filename,cv2.IMREAD_GRAYSCALE) 
print("기존 사진 사이즈")
print(src.shape[1], src.shape[0])

#이미지의 크기를 비율 맞춰 조정.
#기본 수치 744. 안되면 740으로 해보고 그래도 안되면 임의조정.
changed_width = 744
n = (src.shape[1] / changed_width)

changed_height = (src.shape[0] / n)
src = cv2.resize(src , ( changed_width,int(changed_height) ) ) 

print("현재 사진 사이즈")
print(src.shape[1], src.shape[0], "\n")

#양 옆의 노이즈를 제거. 불필요할 시 각주처리할 것.
#src = src[0:src.shape[0]-10, 15:src.shape[1]-25]

#영상 이진화 
ret , binary = cv2.threshold(src,170,255,cv2.THRESH_BINARY_INV) 

#노이즈 제거(camscanner에 의해 어느 정도 보정이 되었으므로 각주 처리)
#binary = cv2.morphologyEx(binary , cv2.MORPH_OPEN , cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2)), iterations = 2) 

#외곽선 검출
contours , hierarchy = cv2.findContours(binary , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
 
#이진화 이미지를 color로 복사한다.
color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

#초록색으로 외곽선을 그려준다.
cv2.drawContours(color , contours , -1 , (0,255,0),3)

"""
검출한 외곽선에 사각형을 그려서 배열에 추가. 이때 크기가 작은 노이즈는 1차로 거른다.
boundingRect : 외각선을 감싸는 최소 크기 사각형 반환. x, y는 좌표. w는 폭, h는 넓이.
append : 리스트에 요소 추가
"""

bR_arr = [] 

for i in range(len(contours)) : 
  bin_tmp = binary.copy() 
  x,y,w,h = cv2.boundingRect(contours[i])
  if x and y > 0 : 
      bR_arr.append([x,y,w,h])

"""
!와 ?의 점까지 인식해보자 프로젝트.
막대기 좌표에 점의 좌표를 더하기!
"""

# 1) 네모가 생길 위치 좌표 저장. 물음표와 느낌표를 찾을때의 x축 범위.
erase_dot = []
count = 0
county = 0

# x를 기준으로 정렬
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

# 네모가 그려질 위치를 미리 파악한다.
for x,y,w,h in bR_arr : 
  tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
  tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산
  # %는 크기를 넓게 세팅.
  # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
  # 그 이외는 글자 크기에 맞게 세팅.
  if (tmp_x and tmp_y > 20) or county == 5 or county == 7: 
      erase_dot.append([x,y]) 
      count += 1

      if count == 10 : 
          count = 0
          county += 1

#100개 들어왔으면 성공. 아니면 이미지 크기를 다시 조정할 것.
print("현재 인식 수치")
print(len(erase_dot))

# 2) 막대기와 점을 연결하자!

# 적용된 문자 수 판별
count = 0

# 1은 느낌표 범위
erase_dot = sorted(erase_dot, key=lambda num : num[0], reverse = False)
erase_dot_x_1 = erase_dot[9][0]
print("느낌표 적용범위", erase_dot_x_1)

# 느낌표. 2번 반복!
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

for n in range(0, 2) : 
    for i in range(0, len(bR_arr)-1) :
        # y축 차이 계산. 배열수 감소로 에러 시 break하고 다음 i로 이동하게 설정.
        for j in range(i+1, len(bR_arr)-1) :
            try : 
                y_difference = int(bR_arr[i][1]) - int(bR_arr[j][1])
            except : 
                break

            if int(bR_arr[i][0] ) <= erase_dot_x_1 and int(bR_arr[j][0]) <= erase_dot_x_1 : 
                # y의 길이 차가 별로 안 날 것.
                if abs(y_difference) < 70 : 

                    # 높이 비교로 막대기 부분 점 부분 판정.
                    if (int(bR_arr[i][3]) < 10) and ( int(bR_arr[j][3]) > 10) :
                        # [i]가 점, [j]가 막대기
                        if int(bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]) > 0 :
                            bR_arr[j][3] = bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1] 

                            if bR_arr[i][0] < bR_arr[j][0] : 
                                bR_arr[j][0] = bR_arr[i][0]
                                bR_arr[j][2] += bR_arr[i][2]

                            del bR_arr[i]
                            count += 1

                    elif (int(bR_arr[i][3]) > 10) and ( int(bR_arr[j][3]) < 10) : 
                        # [i]가 막대기, [j]가 점
                        if int(bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]) > 0:
                            bR_arr[i][3] = bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]

                            if bR_arr[j][0] < bR_arr[i][0] : 
                                bR_arr[i][0] = bR_arr[j][0]
                                bR_arr[i][2] += bR_arr[j][2]

                            del bR_arr[j]
                            count += 1

# 보정 수치.
length = len(erase_dot) - 100

# 2,3은 물음표 범위
for i in range (79+length,len(erase_dot)-1) : 
    if erase_dot[i+1][0] - erase_dot[i][0] >10 :
        erase_dot_x_2 = erase_dot[i+1][0]
        break

for i in range (89+length,len(erase_dot)-1) : 
    if erase_dot[i+1][0] - erase_dot[i][0] >10 :
        erase_dot_x_3 = erase_dot[i+1][0]
        break

print("물음표 적용범위", erase_dot_x_2, erase_dot_x_3)

#물음표. 2번 반복!
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

for n in range(0,2) : 
    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) :
            try : 
                y_difference = int(bR_arr[i][1]) - int(bR_arr[j][1])
            except : 
                break

            if int(bR_arr[i][0] ) >= erase_dot_x_2  and int(bR_arr[j][0]) >= erase_dot_x_2  :
                if int(bR_arr[i][0] ) <= erase_dot_x_3  and int(bR_arr[j][0]) <= erase_dot_x_3  : 

                    # y의 길이 차가 별로 안 날 것.
                    if abs(y_difference) < 70 : 
                        # 높이 비교로 막대기 부분 점 부분 판정.

                        if (int(bR_arr[i][3]) < 10) and ( int(bR_arr[j][3]) > 10) :
                            # [i]가 점, [j]가 막대기
                        
                            if int(bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]) > 0 :
                                bR_arr[j][3] = bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]

                                if bR_arr[i][0] < bR_arr[j][0] : 
                                    bR_arr[j][0] = bR_arr[i][0]
                                    bR_arr[j][2] += bR_arr[i][2]

                                del bR_arr[i]
                                count += 1

                        elif (int(bR_arr[i][3]) > 10) and ( int(bR_arr[j][3]) < 10) :                        
                            # [i]가 막대기, [j]가 점

                            if int(bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]) > 0:
                                bR_arr[i][3] = bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]

                                if bR_arr[j][0] < bR_arr[i][0] : 
                                    bR_arr[i][0] = bR_arr[j][0]
                                    bR_arr[i][2] += bR_arr[j][2]

                                del bR_arr[j]
                                count += 1

print("느낌표 배열")
print(bR_arr[:10])

print("물음표 배열")
for i in range(0, len(bR_arr)-1) : 
    if int(bR_arr[i][0] ) >= erase_dot_x_2 -1 and int(bR_arr[i][0] ) <= erase_dot_x_3 +1:
        print(bR_arr[i])

print("느낌표 물음표 보정 적용 수 : ", count)

"""

 글자 @와 % 를 보정하자.
 직사각형을 그릴 시, 특수문자 전체만이 아닌 일부까지 인식하는 것을 막도록 노이즈 배열을 제거한다.
 큰 네모와 작은 네모가 있을 때 x 좌표와 y 좌표를 비교 후(x_big < x_small 이고 y_big < y_small 일 것. x와 y를 비교)
 작은 네모가 큰 네모 안에 있는지 비교하고 (x_big_1 > x_small_1 이고 y_big_1 > y_small_1일 것. x+w와 y+h를 비교)
 조건이 맞을 시 작은 네모로 판정된 배열을 제거한다.

"""
#y로 정렬한 뒤 시작.
bR_arr = sorted(bR_arr, key=lambda num : num[1], reverse = False)

for n in (0,2) : 
    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) : 

        #베열 파악 중 오류 발생 시 j for문을 break 후 다음 i로 넘기도록 세팅.
        # i가 큰 네모, j가 작은 네모라 가정 후 판단
            try : 
                x_big = int(bR_arr[i][0]) - 10
                y_big = int(bR_arr[i][1]) - 10
                x_small = int(bR_arr[j][0]) 
                y_small = int(bR_arr[j][1])
            except : 
                break

            if x_big < x_small and y_big < y_small : 
                x_big_1 = int(bR_arr[i][0] + bR_arr[i][2]) + 10
                y_big_1 = int(bR_arr[i][1] + bR_arr[i][3]) + 10
                x_small_1 = int(bR_arr[j][0] + bR_arr[j][2])
                y_small_1 = int(bR_arr[j][1] + bR_arr[j][3])

                if x_big_1 > x_small_1 and y_big_1 > y_small_1 :
                    del bR_arr[j]

        # j가 큰 네모, i가 작은 네모라 가정 후 판단
            x_big = int(bR_arr[j][0]) - 10
            y_big = int(bR_arr[j][1]) - 10
            x_small = int(bR_arr[i][0]) 
            y_small = int(bR_arr[i][1])
            
            if x_big < x_small and y_big < y_small : 
                x_big_1 = int(bR_arr[j][0] + bR_arr[j][2]) + 10
                y_big_1 = int(bR_arr[j][1] + bR_arr[j][3]) + 10
                x_small_1 = int(bR_arr[i][0] + bR_arr[i][2])
                y_small_1 = int(bR_arr[i][1] + bR_arr[i][3])

                if x_big_1 > x_small_1 and y_big_1 > y_small_1 :
                    del bR_arr[i]

# 다시 x축 정렬 후 확인
print("최종적으로 남은 배열 수")
print(len(bR_arr))

bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
print(bR_arr[40:60])

"""
# 데이터에 사각형그리기. 10개씩 리스트로 묶어서 저장 
# 순서는 !부터 위에서 아래로 10번(count). 그 뒤에 다음 이모티콘을 저장하는 방식.(county)
# countbox는 네모가 쳐진 갯수. 100개면 성공.
"""
digit_arr = [] 
digit_arr2 = [] 

count = 0
county = 0
countbox = 0

bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

for x,y,w,h in bR_arr : 
  tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
  tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산

  # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
  # %,는 크기를 넓게 세팅. !와 ?는 x축만 조금 보정.
  # 그 이외는 글자 크기에 맞게 세팅.
  if (tmp_x and tmp_y > 20) or county == 5 or county == 7: 
      if county == 0 or county == 8: 
          cv2.rectangle(color,(x-4,y-2),(x+w+4,y+h+2),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-2:y+h+2,x-4:x+w+4]) 
      elif county == 4 : 
          cv2.rectangle(color,(x-10,y-10),(x+w+10,y+h+10),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-10:y+h+10,x-10:x+w+10]) 
      else : 
          cv2.rectangle(color,(x-2,y-2),(x+w+2,y+h+2),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-2:y+h+2,x-2:x+w+2]) 
      
      count += 1
      countbox += 1

      if count == 10 : 
          digit_arr2.append(digit_arr) 
          digit_arr = [] 
          count = 0
          county += 1

#결과 확인
cv2_imshow(color) 
k = cv2.waitKey(0) 
cv2.destroyAllWindows()

print(countbox)

In [ ]:
#리스트에 저장된 이미지를 32x32의 크기로 리사이즈해서 순서대로 저장 
for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
     
    # !과 ?의 경우 비율 유지를 위한 마스크 생성
     if i == 0 or i==8: 
         width = digit_arr2[i][j].shape[1]
         height = digit_arr2[i][j].shape[0] 
         tmp = (height - width)/2 
         mask = np.zeros((height,height)) 
         mask[0:height,int(tmp):int(tmp)+width] = digit_arr2[i][j]
         digit_arr2[i][j] = cv2.resize(mask,(32,32)) 
     else:
         digit_arr2[i][j] = cv2.resize(digit_arr2[i][j],(32,32))

for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
      cv2.imwrite('/content/drive/MyDrive/colab/Minst_test/data/training/'+str(i)+'_'+str(j)+'.png',digit_arr2[i][j])

In [ ]:
데이터 모델 제작 파트

In [ ]:
#데이터 가공
import os 
import cv2
import numpy as np
from sklearn.preprocessing import * 

def createdataset(directory):       #sklearn사용을 위해 데이터세트를 생성
    files = os.listdir(directory)
    x = []
    y = []
    for file in files:
        attr_x = cv2.imread(directory+file, cv2.IMREAD_GRAYSCALE)
        ret , attr_x = cv2.threshold(attr_x,170,255,cv2.THRESH_BINARY) #영상 이진화
        attr_x = attr_x.reshape(32,32)
        attr_y = int(file[0])
        
        x.append(attr_x)
        y.append(attr_y)
        
    x = np.array(x)
    

#    y = np.array([[i] for i in y])
 #   enc = OneHotEncoder(categories='auto')
  #  enc.fit(y) 
   # y = enc.transform(y).toarray()
        
    return x , y
train_dir = '/content/drive/MyDrive/colab/Minst_test/data/training/'
train_images ,train_labels = createdataset(train_dir)

test_dir = '/content/drive/MyDrive/colab/Minst_test/data/test/'
test_images , test_labels = createdataset(test_dir)

print('data set shape :',train_images.shape,len(train_labels),test_images.shape,len(test_labels))

# 라벨 설정
class_names = ['!', '@', '#', '$', '%',
               '^', '&', '*', '?', '+']
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

train_images = train_images / 255.0
test_images = test_images / 255.0

# 데이터 셋트 확인
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
#예제 1
import os
import tensorflow as tf
from tensorflow import keras

# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Flatten(input_shape=(32, 32)),
    keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

 #모델 검증
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

In [ ]:
# 3. 딥러닝 결과 확인
# 딥러닝에 필요한 케라스 함수 호출
from keras.models import load_model
from keras.utils import np_utils

# 필요 라이브러리 호출(PIL은 이미지파일 처리위함)
from PIL import Image
import numpy as np

# test.png는 그림판에서 그린 이미지 파일. 이미지가 있는 경로를 아래 url위치에 넣어주세요
# test.png 파일을 열어서 L(256단계 흑백이미지)로 변환
img = Image.open("/content/drive/MyDrive/colab/Minst_test/test.png").convert("L")

# 변환된 이미지 보기
plt.imshow(img)
plt.show

# 데이터를 모델에 적용할 수 있도록 가공
img = np.resize(img, (100, 32, 32))
test_data = ((np.array(img) / 255) - 1) * -1

# 모델 불러오기
model = load_model('saved_model/my_model')

# 클래스 예측 함수에 가공된 테스트 데이터 넣어 결과 도출
res = np.argmax(model.predict(test_data), axis=-1)
print("결과는 : ", class_names[res[0]])

참고용 코드

In [ ]:
#예제2. 참고용으로만 보시면 도비니다.
#모델 저장 기능이 없습니다.

import os 
import cv2
import numpy as np
from sklearn.preprocessing import * 

def createdataset(directory):       #sklearn사용을 위해 데이터세트를 생성
    files = os.listdir(directory)
    x = []
    y = []
    for file in files:
        attr_x = cv2.imread(directory+file, cv2.IMREAD_GRAYSCALE)
        ret , attr_x = cv2.threshold(attr_x,170,255,cv2.THRESH_BINARY) #영상 이진화
        attr_x = attr_x.reshape(32,32,1)
        attr_y = int(file[0])
        
        x.append(attr_x)
        y.append(attr_y)
        
    x = np.array(x)
    
    y = np.array([[i] for i in y])
    enc = OneHotEncoder(categories='auto')
    enc.fit(y) 
    y = enc.transform(y).toarray()
        
    return x , y
train_dir = '/content/drive/MyDrive/colab/Minst_test/data/training/'
train_x ,train_y = createdataset(train_dir)

test_dir = '/content/drive/MyDrive/colab/Minst_test/data/test/'
test_x , test_y = createdataset(test_dir)

print('data set shape :',train_x.shape,train_y.shape,test_x.shape,test_y.shape)

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# tensorflow는 1.x 버전을 사용합니다. 나중에 명령어들 2.0버전으로 싸그리 교체할 것!
import random 
import matplotlib.pyplot as plt 

# CNN 3층 모델 구현 파트
tf.set_random_seed(777) #reproducibility 

X = tf.placeholder(tf.float32, [None, 32,32,1]) 
Y = tf.placeholder(tf.float32, [None, 10]) 
keep_prob = tf.placeholder(tf.float32) 

#=============================================================================== 
W1 = tf.Variable(tf.random_normal([3, 3, 1, 16], stddev=0.01)) 
#표준편차가 0.01인 3x3매트릭스(깊이1) 데이터를 16개만큼 뽑겠다 
#32x32x16 

L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME') 
#strides를 통해 축을 여러개 만들어 주어 이동방향을 여러방향으로 하게 했으나 가운데 1,1만 적용이 된다 
#제로패딩을통해 사이즈를 동일하게 만듬 

L1 = tf.nn.relu(L1) #relu함수를 이용 음수값을 제거하는 방법으로 사용된다. 

L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
#16x16x16 
#컨벌루션된 결과를 max pooling을 통해 최댓값으로 다시 정리 1차원축으로 2칸 2차원축으로도 2칸이동한다 
#제로패딩을 통해 크기가 줄지 않아야하지만 2x2필터를 사용하여서 크기가 반으로 줄어든다 depth는 그대로 유지된다(32) 

#=============================================================================== 

W2 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01)) 
#역시 필터를 생성하는데 3x3매트릭스의 깊이가16인 데이터를 32개를 사용한다 
#16x16x32

L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2) 
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

#8x8x32 
#=============================================================================== 
#레이어 추가 부분 

W3 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev =0.01)) 
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME') #7x7x64 
L3 = tf.nn.relu(L3) 
L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

#4x4x64 
#=============================================================================== 
L3_flat = tf.reshape(L3,[-1, 4 * 4 * 64]) 

W4 = tf.Variable(tf.random_normal([4 * 4 * 64, 10],stddev=0.01)) 
# 최종 출력값 L2 에서의 출력 4*4*64개를 입력값으로 받아서 0~9 레이블인 10개의 출력값을 만듬 
#=============================================================================== 
b = tf.Variable(tf.random_normal([10])) 
hypothesis = tf.matmul(L3_flat, W4) +b 
target1 = tf.argmax(tf.nn.softmax(hypothesis),1) 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y)) 

learning_rate = 0.001   #러닝하면서 찾아갈 이동 단위

#costfunction 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) 
#adamoptimizer를 사용 adam은 최적값의 근사치일때 넓게 점프해서 한번더 체크해준다 
sess = tf.Session() 
#세션을 통해 메모리에 설정한것들을 올려준다 
sess.run(tf.global_variables_initializer())

#훈련 파트
training_epochs = 20   #훈련을 시킬 횟수
batch_size = 64        #훈련시킬 데이터의 묶음

for epoch in range(training_epochs) :   #같은 트레이닝데이터를 epoch수만큼 훈련시킨다
    avg_cost = 0                        #오버피팅을 통한 정확도 상승을 목표로 한다
    total_batch = int(train_x.shape[0]/ batch_size)
    #train set에 example들을 batchsize(100)만큼 나누어주어 total batch를 구한다
    batch_start = 0
    batch_finish = batch_size
    
    for  i in range(total_batch) :      #전체 데이터를 100장씩 훈련시키게 된다
        batch_xs = train_x[batch_start:batch_finish]
        batch_ys = train_y[batch_start:batch_finish]
        feed_dict = {X:batch_xs, Y:batch_ys}    #placeholder로 설정한 그릇에 데이터 담아줌
        cost_val, _ ,hypo,tg = sess.run([cost,optimizer,hypothesis,target1],feed_dict=feed_dict)
        avg_cost += cost_val / total_batch      #출력을 위해 avg코스트를 만들어주어 확인
        
        batch_start += total_batch
        batch_finish += total_batch
        
        if batch_finish > train_x.shape[0] :
            batch_finish = -1
        
    print('Epoch :', '%04d'%(epoch+1),'cost = ','{:9f}'.format(avg_cost))
    
    
print('Learning Finished!!')        #5번의 훈련(epoch)을 모두 마치면 출력

correct_prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('Accuracy : ', sess.run(accuracy, feed_dict={X:test_x,
                                                   Y:test_y}))

r = random.randint(0,test_x.shape[0]-1)
print('Label : ',sess.run(tf.argmax(test_y[r:r+1],1)))
print('Prediction : ',sess.run(tf.argmax(hypothesis,1),
                               feed_dict={X:test_x[r:r+1]}))

plt.imshow(test_x[r:r+1].reshape(32,32),cmap = 'Greys',
           interpolation='nearest')
plt.show()


Mnist ver(참고용)

In [ ]:
#2. 데이터 학습(아직 데이터 셋이 완성되지 않았으므로 mnist로 테스트.)

#데이터 갯수 설정, mnist import
from keras.datasets import mnist
(X_train, Y_class_train), (X_test, Y_class_test) = mnist.load_data()
print("학습셋 이미지 수 : %d 개" % (X_train.shape[0]))
print("테스트셋 이미지 수 : %d 개" % (X_test.shape[0]))

# 이미지 형태 출력을 위한 pyplot 모듈 import
import sys
import matplotlib.pyplot as plt

# 위 60000개 데이터 배열에서 0번째 원소를 흑백으로 출력
plt.imshow(X_train[0], cmap='Greys')
plt.show()

# 0번째 원소의 모든 데이터를 출력
for x in X_train[0]:
    for i in x:
        sys.stdout.write('%d  ' % i)
    sys.stdout.write('\n')

# 딥러닝에 필요한 케라스 함수 호출
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense

# 필요 라이브러리 호출
import numpy
import tensorflow as tf

# 데이터 셋 호출
from keras.datasets import mnist

# 실행 시마다 같은 결과값 도출을 위한 시드 설정
numpy.random.seed(0)
tf.random.set_seed(0)

# 데이터를 불러와서 각 변수에 저장
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# 학습에 적합한 형태로 데이터 가공
X_train = X_train.reshape(X_train.shape[0], 784).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32') / 255

# 클래스를 학습에 이용하기 위해 데이터 가공
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

# 딥러닝 모델 구조 설정(2개층, 512개의 뉴런 연결, 10개 클래스 출력 뉴런, 784개 픽셀 input 값, relu와 softmax 활성화 함수 이용)
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 딥러닝 구조 설정(loss 옵션을 다중 클래스에 적합한 categorical_crossentropy, 옵티마이저는 adam 설정)
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

# 모델 실행(X_test, Y_test로 검증, 200개씩 30번 학습)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=200, verbose=2)

# 학습 정확도, 검증 정확도 출력
print('\nAccuracy: {:.4f}'.format(model.evaluate(X_train, Y_train)[1]))
print('\nVal_Accuracy: {:.4f}'.format(model.evaluate(X_test, Y_test)[1]))

# 모델 저장
model.save('Predict_Model.h5')

In [ ]:
# 3. 딥러닝 결과 확인
# 딥러닝에 필요한 케라스 함수 호출
from keras.models import load_model
from keras.utils import np_utils

# 필요 라이브러리 호출(PIL은 이미지파일 처리위함)
from PIL import Image
import numpy as np

# test.png는 그림판에서 붓으로 숫자 8을 그린 이미지 파일. 이미지가 있는 경로를 아래 url위치에 넣어주세요
# test.png 파일을 열어서 L(256단계 흑백이미지)로 변환
img = Image.open("/content/drive/MyDrive/colab/Minst_test/test.png").convert("L")

# 변환된 이미지 보기
plt.imshow(img)
plt.show

# 이미지를 784개 흑백 픽셀로 사이즈 변환
img = np.resize(img, (1, 784))

# 데이터를 모델에 적용할 수 있도록 가공
test_data = ((np.array(img) / 255) - 1) * -1

# 모델 불러오기
model = load_model('Predict_Model.h5')

# 클래스 예측 함수에 가공된 테스트 데이터 넣어 결과 도출
res = np.argmax(model.predict(test_data), axis=-1)
print(res)